<a href="https://colab.research.google.com/github/AnastasiyaRahulina/ML_from_scratch/blob/main/Trees_from_skratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import Counter

In [ ]:
class Node:
  def __init__(self, feature=None, treshold=None, left=None, right=None,*, value=None):
    self.feature = feature
    self.treshold = treshold
    self.left = left
    self.right =  right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None

class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_features = n_features
    self.root = None


  def fit(self, X, y):
    self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
    self.root = self._grow_tree(X, y)

  def _grow_tree(self, X, y, current_depth=0):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))

    #check stopping criteria
    if current_depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feature_idxs = np.random.choice(n_features, self.n_features, replace=False)

    #find the best split
    best_feature, best_treshold = self._best_split(X, y, feature_idxs)

    #create child nodes
    left_idxs, right_idxs = self._split(X[:, best_feature], best_treshold)
    left = self._grow_tree(X[left_idxs,:], y[left_idxs], current_depth+1)
    right = self._grow_tree(X[right_idxs,:], y[right_idxs], current_depth+1)
    return Node(best_feature, best_treshold, left, right)

  def _best_split(self, X, y, feature_idxs):
    best_gain = -1
    split_idx, split_treshold = None, None
    for feature_idx in feature_idxs:
      X_column = X[:, feature_idx]
      tresholds = np.unique(X_column)
      for thr in tresholds:
        #calculate information gain
        gain = self._information_gain(y, X_column, thr)
        if gain > best_gain:
          best_gain = gain
          split_idx = feature_idx
          split_treshold = thr
    return split_idx, split_treshold

  def _information_gain(self, y, X_column, treshold):
    #parent entropy
    parent_entropy = self._entropy(y)

    #create children
    left_idxs, right_idxs = self._split(X_column, treshold)
    if len(left_idxs)==0 or len(right_idxs)==0:
      return 0

    #calculate the weighted avg entropy of the children
    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

    return parent_entropy - child_entropy


    #calculate the information gain

  def _split(self, X_column, treshold):
    left_idxs = np.argwhere(X_column<=treshold).flatten()
    right_idxs = np.argwhere(X_column>treshold).flatten()
    return left_idxs, right_idxs


  def _entropy(self, y):
    histogram = np.bincount(y)
    ps = histogram / len(y)
    return -np.sum([p* np.log(p) for p in ps if p>0])


  def _most_common_label(self, y):
    return np.argmax(np.bincount(y))

  def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value
    if x[node.feature] <= node.treshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)






In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


In [ ]:
data = datasets.load_breast_cancer()

X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

def accuracy(y_true, y_pred):
  accuracy = np.sum(y_true == y_pred) / len(y_true)
  return accuracy

acc = accuracy(y_test, predictions)
print(acc)

0.9210526315789473


In [ ]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_feature=None):
        self.n_trees = n_trees
        self.max_depth=max_depth
        self.min_samples_split=min_samples_split
        self.n_features=n_feature
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,
                            min_samples_split=self.min_samples_split,
                            n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        predictions = np.array([self._most_common_label(pred) for pred in tree_preds])
        return predictions





In [ ]:
data = datasets.load_breast_cancer()

X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

clf = RandomForest()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
acc = accuracy(y_test, predictions)
print(acc)

0.9649122807017544
